In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import datetime

In [5]:
filehandler = open("transactions_6mth.pkl", "rb")
df_trans = pickle.load(filehandler)
filehandler.close()

In [14]:
day_start = datetime.datetime(2020, 9, 9, 0, 0)
day_end = datetime.datetime(2020, 9, 15, 0, 0)
N = 12

q_date = ""
q_date = f"((t_dat >= '{day_start}') and (t_dat <= '{day_end}'))"

top_art_all = df_trans.query(q_date).groupby("article_id")["t_dat"].count().sort_values(ascending = False).index[:N].tolist()

# Fake transations
customer_array = np.repeat(df_trans.customer_id.unique(), N)
article_array = np.tile(top_art_all, len(df_trans.customer_id.unique()))
df_setup = pd.DataFrame({'customer_id':customer_array, 'article_id':article_array})



In [16]:
article_rec = df_setup.groupby("customer_id").article_id.apply(lambda x: list(x)[:12])
rec_dict = article_rec.to_dict()

In [19]:
# Start validation

test = transactions_6mth.loc[transactions_6mth.t_dat >= '2020-09-16',:]

In [20]:
def calc_cust_precision(u_id):
    ground_truth = cust_orders[u_id]
    hits_list = (np.isin(rec_dict[u_id], ground_truth))*1
    ap = 0 ## average precision
    hits = np.sum(hits_list)
    relevant_n = len(ground_truth)

    if (hits != 0):
        for rank in range(1,len(hits_list)+1):
            # update ap on hit items
            hit = hits_list[rank-1]
            if hit:
                rank_hit_list = hits_list[0:rank]
                prec_i = np.mean(rank_hit_list)
                ap += prec_i  
        ap /= (min(relevant_n, 12))
    
    return ap

In [21]:
cust_orders = test.groupby("customer_id")["article_id"].unique()

In [22]:
customer_list = test.customer_id.unique()
ap_u = 0.

for u_id in customer_list:
    ap_u += calc_cust_precision(u_id)
mean_ap = ap_u / len(customer_list) 

In [23]:
mean_ap

0.013036469248279512